# Import

In [ ]:
import os
import gc
import time
start_time = time.time()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
puns = string.punctuation

import gensim

# Loading Data

In [ ]:
df_train = pd.read_table('../input/mercari-price-suggestion-challenge/train.tsv')
df_test = pd.read_table('../input/mercari-price-suggestion-challenge/test.tsv')
# for testing(concerning time)
# df_train = pd.read_table('../input/mercari-price-suggestion-challenge/train.tsv').loc[:10000]
# df_test = pd.read_table('../input/mercari-price-suggestion-challenge/test.tsv').loc[:10000]
df_train['target'] = np.log1p(df_train['price']) ## (np.log(array + 1))

In [ ]:
print(df_train.shape)
print('5 folds scaling the test_df')
print(df_test.shape)
test_len = df_test.shape[0]
def simulate_test(df_test):
    if df_test.shape[0] < 800000:
        indices = np.random.choice(df_test.index.values, 2800000)
        df_test_ = pd.concat([df_test, df_test.iloc[indices]], axis=0)
        return df_test_.copy()
    else:
        return df_test
df_test = simulate_test(df_test)

print('new shape ', df_test.shape)
print('[{}] Finished scaling test set...'.format(time.time() - start_time))

# Handling missing values

In [ ]:
print("Handling missing values...")
def handle_missing(dataset):
    dataset.category_name.fillna(value="missing", inplace=True)
    dataset.brand_name.fillna(value="missing", inplace=True)
    dataset.item_description.fillna(value="missing", inplace=True)
    return (dataset)

df_train = handle_missing(df_train)
df_test = handle_missing(df_test)
print(df_train.shape)
print(df_test.shape)

print('[{}] Finished handling missing data...'.format(time.time() - start_time))

# Preprocess - Word2Vec
considering time, this part is abandomed

In [ ]:
# def tokenize(Doc):
#     if pd.notnull(Doc):
#         tokens = nltk.wordpunct_tokenize(Doc)
#         text = nltk.Text(tokens)
#         words = [w.lower() for w in text if w not in stops and w not in puns]
#         return words
#     else:
#         return None

In [ ]:
# tokens = []
# for sent in np.hstack([df_train['item_description'], df_test['item_description'], 
#            df_train['name'], df_test['name']]):
#     if pd.notnull(sent):
#         tokens.extend(tokenize(sent))

# tokens = list(set(tokens))

In [ ]:
# word_vec_mapping = {}
# path = "../input/wrod2vec-twitter-50d/glove.twitter.27B.50d.txt"

# with open(path, 'r', encoding='utf8') as f:
#     for line in f:
#         tokens = line.split()
#         token = tokens[0]
#         vec = tokens[1:]
#         if token in tokens:
#             word_vec_mapping[token] = np.array(vec, dtype=np.float32)
# vec_dimensions = len(word_vec_mapping.get('a'))

In [ ]:
# def doc2vec(doc, word2vec=word_vec_mapping):
#     docvec=np.zeros(vec_dimensions, )
#     vec_count = 1
    
#     if pd.notnull(doc):
#         terms = tokenize(doc)
#         for term in terms:
#             termvec = word_vec_mapping.get(term, None)
#             if termvec is not None:
#                 docvec += np.array(termvec, dtype=np.float32)
#                 vec_count += 1              
#     return (docvec/vec_count)

# def doc2vec_desc(doc, word2vec=word_vec_mapping):
#     docvec=np.zeros(vec_dimensions, )
#     vec_count = 1
    
#     if pd.notnull(doc):
#         terms = tokenize(doc)
#         for term in terms:
#             termvec = word_vec_mapping.get(term, None)
#             if termvec is not None:
#                 docvec += np.array(termvec, dtype=np.float32)
#                 vec_count += 1              
#     return (docvec/vec_count)

# def embedding_name_cat_word2vec(df):
#     name_vecs = []
#     desc_vecs = []
#     for idx, row in df.iterrows():
#         name_vecs.append(doc2vec(row['name']))
#         desc_vecs.append(doc2vec(row['item_description']))
#         if idx % 200000 == 0:
#             print(idx)
#     df['name_word2vec'] = name_vecs
#     df['item_description_word2vec'] = desc_vecs
#     return df

# df_train = embedding_name_cat_word2vec(df_train)
# df_test = embedding_name_cat_word2vec(df_test)

# Peprocess - category and brand

In [ ]:
#PROCESS CATEGORICAL DATA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, MultiLabelBinarizer
print("Handling categorical variables...")
le_cat = LabelEncoder()
le_cat.fit(np.hstack([df_train.category_name, df_test.category_name]))
df_train['category'] = le_cat.transform(df_train.category_name)
df_test['category'] = le_cat.transform(df_test.category_name)

le_brand = LabelEncoder()
le_brand.fit(np.hstack([df_train.brand_name, df_test.brand_name]))
df_train['brand'] = le_brand.transform(df_train.brand_name)
df_test['brand'] = le_brand.transform(df_test.brand_name)
# del le, df_train['brand_name'], df_test['brand_name']

print('[{}] Finished PROCESSING CATEGORICAL DATA...'.format(time.time() - start_time))
df_train.head(3)

# Peprocess - category_split

In [ ]:
#Build Category LabelEncoder
def serialize_category(cats):
    return cats.split('/')
df_train['category_split'] = df_train['category_name'].apply(serialize_category)
df_test['category_split'] = df_test['category_name'].apply(serialize_category)
mlb = MultiLabelBinarizer()    
mlb.fit(np.hstack([df_train['category_split'], df_test['category_split']]))

idx_cat_mapping = {}
for idx, cat_name in enumerate(mlb.classes_):
    idx_cat_mapping[cat_name] = idx

def getidxs(cats):
    idxs = []
    for cat in cats:
        idx = idx_cat_mapping.get(cat)
        idxs.append(idx)
    return idxs

df_train['category_split'] = df_train['category_split'].apply(getidxs)
df_test['category_split'] = df_test['category_split'].apply(getidxs)

# Prprocess - category_name_onehot, item_description_onehot, name_onehot

In [ ]:
#PROCESS TEXT: RAW
print("Text to seq process...")
print("   Fitting tokenizer...")
from keras.preprocessing.text import Tokenizer
raw_text = np.hstack([df_train['category_name'].str.lower(), 
                      df_train['item_description'].str.lower(), 
                      df_train['name'].str.lower(),
                      df_test['category_name'].str.lower(), 
                      df_test['item_description'].str.lower(), 
                      df_test['name'].str.lower()])

tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)
print("   Transforming text to seq...")
df_train["category_name_onehot"] = tok_raw.texts_to_sequences(df_train['category_name'].str.lower())
df_test["category_name_onehot"] = tok_raw.texts_to_sequences(df_test['category_name'].str.lower())
df_train["item_description_onehot"] = tok_raw.texts_to_sequences(df_train['item_description'].str.lower())
df_test["item_description_onehot"] = tok_raw.texts_to_sequences(df_test['item_description'].str.lower())
df_train["name_onehot"] = tok_raw.texts_to_sequences(df_train['name'].str.lower())
df_test["name_onehot"] = tok_raw.texts_to_sequences(df_test['name'].str.lower())
df_train.head(3)

print('[{}] Finished PROCESSING TEXT DATA...'.format(time.time() - start_time))

# train_test_split

In [ ]:
#EXTRACT DEVELOPTMENT TEST
from sklearn.model_selection import train_test_split
dtrain, dvalid = train_test_split(df_train, random_state=233, train_size=0.99)
print(dtrain.shape)
print(dvalid.shape)

# Preparing Keras Input

In [ ]:
#EMBEDDINGS MAX VALUE
#Base on the histograms, we select the next lengths
MAX_NAME_SEQ = 20 #17
MAX_ITEM_DESC_SEQ = 60 #269
MAX_CATEGORY_NAME_SEQ = 20 #8
MAX_CATEGORY_SPLIT_SEQ = np.max([df_train['category_split'].apply(len).max(), df_test['category_split'].apply(len).max()])

MAX_TEXT = len(tok_raw.word_index) +1
MAX_CATEGORY = len(le_cat.classes_)
MAX_CATEGORY_SPLIT = len(mlb.classes_)
MAX_BRAND = len(le_brand.classes_)
MAX_CONDITION = np.max([df_train.item_condition_id.max(), 
                        df_test.item_condition_id.max()])+1

print('[{}] Finished EMBEDDINGS MAX VALUE...'.format(time.time() - start_time))

In [ ]:
#KERAS DATA DEFINITION
from keras.preprocessing.sequence import pad_sequences

def get_keras_data(dataset):
    X = {
        'name_onehot': pad_sequences(dataset['name_onehot'], maxlen=MAX_NAME_SEQ),
        'item_description_onehot': pad_sequences(dataset['item_description_onehot']
                                    , maxlen=MAX_ITEM_DESC_SEQ),
        'brand': np.array(dataset['brand']),
        'category': np.array(dataset['category']),
        'category_split': pad_sequences(dataset.category_split, maxlen=MAX_CATEGORY_SPLIT_SEQ),
        'category_name_onehot': pad_sequences(dataset['category_name_onehot']
                                        , maxlen=MAX_CATEGORY_NAME_SEQ),
        'item_condition': np.array(dataset.item_condition_id),
        'num_vars': np.array(dataset[["shipping"]])
    }
    return X

X_train = get_keras_data(dtrain)
X_valid = get_keras_data(dvalid)
X_test = get_keras_data(df_test)

print('[{}] Finished DATA PREPARARTION...'.format(time.time() - start_time))

# Building Keras Model

In [ ]:
#KERAS MODEL DEFINITION
from keras.layers import Input, Dropout, Dense, BatchNormalization, \
    Activation, concatenate, GRU, Embedding, Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping#, TensorBoard
from keras import backend as K
from keras import optimizers
from keras import initializers

def rmsle(y, y_pred):
    import math
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 \
              for i, pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

dr = 0.25

def get_model():
    #params
    dr_r = dr
    
    #Inputs
    name_onehot = Input(shape=[X_train["name_onehot"].shape[1]], name="name_onehot")
    item_description_onehot = Input(shape=[X_train["item_description_onehot"].shape[1]], 
                                    name="item_description_onehot")
    brand = Input(shape=[1], name="brand")
    category = Input(shape=[1], name="category")
    category_split = Input(shape=[X_train["category_split"].shape[1]], name="category_split")
    category_name_onehot = Input(shape=[X_train["category_name_onehot"].shape[1]], 
                          name="category_name_onehot")
    item_condition = Input(shape=[1], name="item_condition")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    
    #Embeddings layers
    emb_size = 60
    
    emb_name_onehot = Embedding(MAX_TEXT, emb_size//3)(name_onehot)
    emb_item_description_onehot = Embedding(MAX_TEXT, emb_size)(item_description_onehot)
    emb_category_name_onehot = Embedding(MAX_TEXT, emb_size//3)(category_name_onehot)
    emb_category_split = Embedding(MAX_CATEGORY_SPLIT, emb_size//2)(category_split)
    emb_brand = Embedding(MAX_BRAND, 10)(brand)
    emb_category = Embedding(MAX_CATEGORY, 10)(category)
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)
    
    rnn_layer1 = GRU(16) (emb_item_description_onehot)
    rnn_layer2 = GRU(8) (emb_category_name_onehot)
    rnn_layer3 = GRU(8) (emb_category_split)
    rnn_layer4 = GRU(8) (emb_name_onehot)
    
    #main layer
    main_l = concatenate([
        Flatten() (emb_brand)
        , Flatten() (emb_category)
        , Flatten() (emb_item_condition)
        , rnn_layer1
        , rnn_layer2
        , rnn_layer3
        , rnn_layer4
        , num_vars
    ])
    main_l = Dropout(dr)(Dense(512,activation='relu') (main_l))
    main_l = Dropout(dr)(Dense(64,activation='relu') (main_l))
    main_l = Dropout(dr)(Dense(32,activation='relu') (main_l))
    
    #output
    output = Dense(1,activation="linear") (main_l)
    
    #model
    model = Model([name_onehot, item_description_onehot, brand
                   , category, category_name_onehot, category_split
                   , item_condition, num_vars], output)
    #optimizer = optimizers.RMSprop()
    optimizer = optimizers.Adam()
    model.compile(loss="mse", 
                  optimizer=optimizer)
    return model

def eval_model(model):
    val_preds = model.predict(X_valid)
    val_preds = np.expm1(val_preds)
    
    y_true = np.array(dvalid.price.values)
    y_pred = val_preds[:, 0]
    v_rmsle = rmsle(y_true, y_pred)
    print(" RMSLE error on dev test: "+str(v_rmsle))
    return v_rmsle
#fin_lr=init_lr * (1/(1+decay))**(steps-1)
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1

print('[{}] Finished DEFINEING MODEL...'.format(time.time() - start_time))

# Train

In [ ]:
gc.collect()
#FITTING THE MODEL
epochs = 2
BATCH_SIZE = 512 * 3
steps = int(len(X_train['name_onehot'])/BATCH_SIZE) * epochs
lr_init, lr_fin = 0.013, 0.009
lr_decay = exp_decay(lr_init, lr_fin, steps)
log_subdir = '_'.join(['ep', str(epochs),
                    'bs', str(BATCH_SIZE),
                    'lrI', str(lr_init),
                    'lrF', str(lr_fin),
                    'dr', str(dr)])

model = get_model()
K.set_value(model.optimizer.lr, lr_init)
K.set_value(model.optimizer.decay, lr_decay)

history = model.fit(X_train, dtrain.target
                    , epochs=epochs
                    , batch_size=BATCH_SIZE
                    , validation_split=0.01
                    #, callbacks=[TensorBoard('./logs/'+log_subdir)]
                    , verbose=10
                    )

print('[{}] Finished FITTING MODEL...'.format(time.time() - start_time))
#EVLUEATE THE MODEL ON DEV TEST
v_rmsle = eval_model(model)
print('[{}] Finished predicting valid set...'.format(time.time() - start_time))

# Pred

In [ ]:
#CREATE PREDICTIONS
preds = model.predict(X_test, batch_size=BATCH_SIZE)
preds = np.expm1(preds)
print('[{}] Finished predicting test set...'.format(time.time() - start_time))
submission = df_test[["test_id"]]
submission["price"] = preds
submission.to_csv("./myNN"+log_subdir+"_{:.6}.csv".format(v_rmsle), index=False)
print('[{}] Finished submission...'.format(time.time() - start_time))